In [128]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


In [129]:
finance = pd.read_csv('sp500_features.csv')
sentimental = pd.read_csv('reddit_with_vader.csv')
new_sentimental = pd.read_csv('new_reddit_with_vader.csv')

In [130]:
finance.head(5)

,index,Open,High,Low,Close,Adj Close,Volume,ticker,year,month,week_year,date,week_month,OBV,ema,rsi,macd,macd_h,macd_s
0,2013-01-01,30.164520,30.722462,29.434908,30.436338,27.874155,24564398,A,2013,1,1,2013-01-07,1,0.0,30.436338,NaN,NaN,NaN,NaN
1,2013-01-08,30.379112,31.559372,30.114449,31.144491,28.522694,21629576,A,2013,1,2,2013-01-14,2,21629576.0,30.613377,NaN,NaN,NaN,NaN
2,2013-01-15,30.865522,31.781116,30.636623,31.580830,28.922304,17762849,A,2013,1,3,2013-01-21,3,39392425.0,30.855240,NaN,NaN,NaN,NaN
3,2013-01-22,31.444921,32.582260,31.366238,32.317596,29.597048,18998819,A,2013,1,4,2013-01-28,4,58391244.0,31.220829,NaN,NaN,NaN,NaN
4,2013-01-29,32.160229,32.660946,31.387697,31.659513,28.994360,21775387,A,2013,1,5,2013-02-04,1,36615857.0,31.330500,NaN,NaN,NaN,NaN


In [131]:
sentimental.sort_values(by=['ticker','year','month','woy'], inplace=True)
sentimental.head(5)

,Unnamed: 0,year,month,woy,ticker,subreddit,id,created,dups,vader_dict,polarity
257406,257406,2013,1,1,A,stocks,162qib,2013-01-06 17:13:35,0,"{'neg': 0.0, 'neu': 0.858, 'pos': 0.142, 'comp...",neu
305401,305401,2013,1,1,A,business,15teca,2013-01-02 04:45:34,1,"{'neg': 0.101, 'neu': 0.842, 'pos': 0.058, 'co...",neu
305402,305402,2013,1,1,A,business,15su8x,2013-01-01 22:53:16,1,"{'neg': 0.081, 'neu': 0.843, 'pos': 0.076, 'co...",neu
305403,305403,2013,1,1,A,business,15ydj2,2013-01-04 12:57:25,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neu
305404,305404,2013,1,1,A,business,15yaml,2013-01-04 12:04:30,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neu


In [132]:
new_sentimental.head(5)

,Unnamed: 0,year,month,woy,ticker,subreddit,id,created,dups,vader_dict,polarity
0,0,2023,2,5,AAPL,Stock_Picks,10rj4ph,2023-02-02 03:38:17,1,"{'neg': 0.049, 'neu': 0.833, 'pos': 0.118, 'co...",neu
1,1,2023,1,2,AAPL,Stock_Picks,108exkr,2023-01-10 14:06:43,1,"{'neg': 0.0, 'neu': 0.926, 'pos': 0.074, 'comp...",neu
2,2,2023,3,10,AAPL,StockMarket,11kgwgj,2023-03-06 19:51:15,1,"{'neg': 0.023, 'neu': 0.817, 'pos': 0.16, 'com...",neu
3,3,2023,3,10,AAPL,StockMarket,11jykl2,2023-03-06 10:12:04,1,"{'neg': 0.024, 'neu': 0.919, 'pos': 0.057, 'co...",neu
4,4,2023,3,10,AAPL,StockMarket,11jy802,2023-03-06 09:57:51,1,"{'neg': 0.025, 'neu': 0.868, 'pos': 0.108, 'co...",neu


In [133]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(sentimental[['polarity']])
sentimental[ohe.get_feature_names_out()] = ohe.transform(sentimental[['polarity']])

sentimental.drop(columns=['polarity'], inplace=True)
sentimental_ohe = sentimental.groupby(['ticker','year','month','woy'])['polarity_neu','polarity_pos','polarity_neg'].sum()
sentimental_ohe.to_csv('ohe.csv')

/Users/fabrii/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/var/folders/5l/cbmvrvp95ldc5v4qdzr86h5m0000gn/T/ipykernel_43249/730289640.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sentimental_ohe = sentimental.groupby(['ticker','year','month','woy'])['polarity_neu','polarity_pos','polarity_neg'].sum()


In [134]:
sent_ohe = pd.read_csv('ohe.csv')
sent_ohe

,ticker,year,month,woy,polarity_neu,polarity_pos,polarity_neg
0,A,2013,1,1,23.0,5.0,0.0
1,A,2013,1,2,26.0,9.0,1.0
2,A,2013,1,3,20.0,11.0,0.0
3,A,2013,1,4,24.0,7.0,2.0
4,A,2013,1,5,22.0,6.0,1.0
...,...,...,...,...,...,...,...
80944,ZTS,2022,5,20,1.0,0.0,0.0
80945,ZTS,2022,6,24,1.0,0.0,0.0
80946,ZTS,2022,7,30,1.0,0.0,0.0
80947,ZTS,2022,8,31,1.0,0.0,0.0


In [135]:
finance

,index,Open,High,Low,Close,Adj Close,Volume,ticker,year,month,week_year,date,week_month,OBV,ema,rsi,macd,macd_h,macd_s
0,2013-01-01,30.164520,30.722462,29.434908,30.436338,27.874155,24564398,A,2013,1,1,2013-01-07,1,0.000000e+00,30.436338,NaN,NaN,NaN,NaN
1,2013-01-08,30.379112,31.559372,30.114449,31.144491,28.522694,21629576,A,2013,1,2,2013-01-14,2,2.162958e+07,30.613377,NaN,NaN,NaN,NaN
2,2013-01-15,30.865522,31.781116,30.636623,31.580830,28.922304,17762849,A,2013,1,3,2013-01-21,3,3.939242e+07,30.855240,NaN,NaN,NaN,NaN
3,2013-01-22,31.444921,32.582260,31.366238,32.317596,29.597048,18998819,A,2013,1,4,2013-01-28,4,5.839124e+07,31.220829,NaN,NaN,NaN,NaN
4,2013-01-29,32.160229,32.660946,31.387697,31.659513,28.994360,21775387,A,2013,1,5,2013-02-04,1,3.661586e+07,31.330500,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260591,2023-02-06,166.360001,166.369995,157.869995,158.330002,158.330002,9923300,ZTS,2023,2,6,2023-02-06,1,4.075275e+11,159.378219,52.358887,-0.954834,2.952760,-3.907594
260592,2023-02-13,158.139999,176.660004,158.080002,172.029999,172.029999,12298000,ZTS,2023,2,7,2023-02-13,2,4.075398e+11,162.541164,66.820897,0.308841,3.373148,-3.064307
260593,2023-02-20,171.529999,172.080002,163.910004,166.330002,166.330002,7507300,ZTS,2023,2,8,2023-02-20,3,4.075323e+11,163.488374,58.239373,0.840680,3.123989,-2.283310
260594,2023-02-27,168.000000,170.710007,164.990005,170.559998,170.559998,8983000,ZTS,2023,2,9,2023-02-27,4,4.075413e+11,165.256280,62.418091,1.585218,3.094822,-1.509604


In [136]:
finance_new = finance.drop(columns=['index','date','week_month'])
finance_new.rename(columns = {'week_year':'woy'}, inplace = True)
data = finance_new.merge(sent_ohe, how='left')
data

,Open,High,Low,Close,Adj Close,Volume,ticker,year,month,woy,OBV,ema,rsi,macd,macd_h,macd_s,polarity_neu,polarity_pos,polarity_neg
0,30.164520,30.722462,29.434908,30.436338,27.874155,24564398,A,2013,1,1,0.000000e+00,30.436338,NaN,NaN,NaN,NaN,23.0,5.0,0.0
1,30.379112,31.559372,30.114449,31.144491,28.522694,21629576,A,2013,1,2,2.162958e+07,30.613377,NaN,NaN,NaN,NaN,26.0,9.0,1.0
2,30.865522,31.781116,30.636623,31.580830,28.922304,17762849,A,2013,1,3,3.939242e+07,30.855240,NaN,NaN,NaN,NaN,20.0,11.0,0.0
3,31.444921,32.582260,31.366238,32.317596,29.597048,18998819,A,2013,1,4,5.839124e+07,31.220829,NaN,NaN,NaN,NaN,24.0,7.0,2.0
4,32.160229,32.660946,31.387697,31.659513,28.994360,21775387,A,2013,1,5,3.661586e+07,31.330500,NaN,NaN,NaN,NaN,22.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260591,166.360001,166.369995,157.869995,158.330002,158.330002,9923300,ZTS,2023,2,6,4.075275e+11,159.378219,52.358887,-0.954834,2.952760,-3.907594,NaN,NaN,NaN
260592,158.139999,176.660004,158.080002,172.029999,172.029999,12298000,ZTS,2023,2,7,4.075398e+11,162.541164,66.820897,0.308841,3.373148,-3.064307,NaN,NaN,NaN
260593,171.529999,172.080002,163.910004,166.330002,166.330002,7507300,ZTS,2023,2,8,4.075323e+11,163.488374,58.239373,0.840680,3.123989,-2.283310,NaN,NaN,NaN
260594,168.000000,170.710007,164.990005,170.559998,170.559998,8983000,ZTS,2023,2,9,4.075413e+11,165.256280,62.418091,1.585218,3.094822,-1.509604,NaN,NaN,NaN


In [137]:
data['polarity_neg'] = data['polarity_neg'].fillna(0)
data['polarity_neu'] = data['polarity_neu'].fillna(0)
data['polarity_pos'] = data['polarity_pos'].fillna(0)

data.isna().sum()

Open             0
High             0
Low              0
Close            0
Adj Close        0
Volume           0
ticker           0
year             0
month            0
woy              0
OBV              0
ema              0
rsi              7
macd            25
macd_h          33
macd_s          33
polarity_neu     0
polarity_pos     0
polarity_neg     0
dtype: int64

In [138]:
data.to_csv('sentimental_finance_features.csv')